In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


# Introduction to ``torch.compile``
**Author:** William Wen


``torch.compile`` is the latest method to speed up your PyTorch code!
``torch.compile`` makes PyTorch code run faster by
JIT-compiling PyTorch code into optimized kernels,
all while requiring minimal code changes.

In this tutorial, we cover basic ``torch.compile`` usage,
and demonstrate the advantages of ``torch.compile`` over
previous PyTorch compiler solutions, such as
[TorchScript](https://pytorch.org/docs/stable/jit.html)_ and
[FX Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace)_.

**Contents**

**Required pip Dependencies**

- ``torch >= 2.0``
- ``torchvision``
- ``numpy``
- ``scipy``
- ``tabulate``



NOTE: a modern NVIDIA GPU (H100, A100, or V100) is recommended for this tutorial in
order to reproduce the speedup numbers shown below and documented elsewhere.



In [3]:
import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    print(device_cap)
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

(8, 6)


/tmp/ipykernel_44172/2836601257.py:12: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(


## Basic Usage

``torch.compile`` is included in the latest PyTorch.
Running TorchInductor on GPU requires Triton, which is included with the PyTorch 2.0 nightly
binary. If Triton is still missing, try installing ``torchtriton`` via pip
(``pip install torchtriton --extra-index-url "https://download.pytorch.org/whl/nightly/cu117"``
for CUDA 11.7).

Arbitrary Python functions can be optimized by passing the callable to
``torch.compile``. We can then call the returned optimized
function in place of the original function.



In [4]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
print(opt_foo1(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[-5.6378e-01,  1.2836e+00, -1.0063e+00, -1.0048e-01, -1.5855e-01,
          1.2504e+00,  1.5489e+00,  1.3908e+00,  8.6342e-01,  7.7401e-01],
        [-2.8800e-01,  8.1421e-01,  1.4728e+00,  1.0835e-01,  8.5950e-01,
         -7.0951e-02,  1.2286e-03, -4.3115e-01, -8.0697e-01, -7.3548e-02],
        [ 1.8580e+00,  8.7403e-03, -6.1277e-01, -4.9156e-01,  1.5231e+00,
         -1.6085e+00,  1.7979e+00,  5.7933e-01,  1.4469e+00,  1.4111e+00],
        [ 1.4441e+00,  1.4408e+00,  1.6808e-01, -7.2325e-01,  1.1878e+00,
          1.0554e+00, -1.5067e-01,  6.3707e-01, -2.0760e-01,  1.0917e+00],
        [ 9.5867e-01,  1.9794e+00,  1.2888e+00,  1.2698e+00,  5.0365e-01,
          5.0774e-01,  5.5544e-02,  9.8845e-01,  1.0738e+00,  1.2924e+00],
        [ 1.2953e+00, -3.5014e-01, -8.0273e-01, -6.7772e-01,  1.9081e+00,
         -4.9449e-03,  1.4697e+00,  8.3228e-02,  4.7851e-01,  1.2125e+00],
        [ 1.5045e+00,  1.4964e+00, -2.6385e-01,  1.3657e+00,  1.2996e+00,
          1.7351e+00,  1.6556e+0

Alternatively, we can decorate the function.



In [5]:
@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
print(opt_foo2(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 0.7100,  1.4268,  1.6995,  0.3673, -0.0242,  0.5008,  1.1849, -0.0344,
          1.2291, -0.3591],
        [ 0.3391,  0.9448, -1.4141, -0.1750,  1.9362,  1.9697, -0.0164,  0.1766,
          0.5867,  1.4973],
        [ 1.3731,  1.8254,  1.1368,  1.4214,  1.8714, -1.2382,  1.0648,  0.2109,
          1.0688,  1.1489],
        [ 0.9088, -0.4864,  1.3935,  1.9034,  0.2980, -0.3237,  1.6973,  0.9423,
          0.2022,  0.2463],
        [ 0.8164,  0.4968,  1.3048,  1.7325,  1.0092,  1.5822,  0.1893, -0.2870,
         -0.2509, -1.2097],
        [ 0.4151,  1.3024, -0.0914,  0.2478, -0.0190,  1.8713,  0.7877,  1.5712,
          1.4208,  1.8738],
        [ 0.1697, -0.5418,  0.4589,  0.0186, -0.4758, -0.0195,  1.5589, -0.0447,
         -0.7707,  1.3753],
        [-0.1523,  0.3960,  0.8981,  0.2410,  0.1351,  1.7552, -0.7020,  1.5370,
          0.2438,  0.3210],
        [-0.2068,  1.4767,  1.8646,  0.2873, -0.0724,  1.2453,  1.8641, -0.1732,
         -0.4352, -0.0894],
        [ 0.5164, -

We can also optimize ``torch.nn.Module`` instances.



In [6]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
opt_mod = torch.compile(mod)
print(opt_mod(torch.randn(10, 100)))

tensor([[0.0000, 0.2028, 0.0000, 0.0000, 0.0000, 0.3608, 0.0000, 0.0000, 0.1197,
         0.0000],
        [0.0000, 0.9833, 0.7477, 0.8377, 0.2258, 0.1237, 0.7125, 0.1590, 0.0000,
         0.5059],
        [0.0000, 0.0000, 0.1076, 0.0000, 0.0051, 0.0000, 0.0000, 0.3113, 0.0000,
         0.2658],
        [0.5126, 0.1726, 0.4858, 0.0000, 0.4324, 0.4479, 0.4244, 0.0000, 0.0000,
         0.8051],
        [0.0000, 0.6025, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3658, 0.0000,
         0.0000],
        [0.0000, 0.2222, 0.3405, 1.4069, 0.0000, 0.5194, 0.1953, 0.0000, 0.0000,
         0.0000],
        [0.1542, 0.6048, 0.6657, 0.0000, 0.0217, 0.2964, 0.5083, 0.1983, 0.0000,
         0.8327],
        [1.1459, 0.7917, 0.8754, 0.0000, 0.1485, 0.0000, 0.2702, 0.0000, 0.0000,
         0.0000],
        [0.7501, 0.0000, 1.3762, 0.1484, 0.6177, 0.0000, 0.0000, 0.0000, 0.6045,
         0.1895],
        [0.2181, 0.1413, 0.0000, 0.0000, 0.0000, 0.0000, 0.6448, 0.4287, 1.1657,
         -0.0000]], grad_fn=

## Demonstrating Speedups

Let's now demonstrate that using ``torch.compile`` can speed
up real models. We will compare standard eager mode and
``torch.compile`` by evaluating and training a ``torchvision`` model on random data.

Before we start, we need to define some utility functions.



In [7]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

N_ITERS = 10

from torchvision.models import densenet121
def init_model():
    return densenet121().to(torch.float32).cuda()

First, let's compare inference.

Note that in the call to ``torch.compile``, we have have the additional
``mode`` argument, which we will discuss below.



In [8]:
model = init_model()

# Reset since we are using a different mode.
import torch._dynamo
torch._dynamo.reset()

model_opt = torch.compile(model, mode="reduce-overhead")

inp = generate_data(16)[0]
with torch.no_grad():
    print("eager:", timed(lambda: model(inp))[1])
    print("compile:", timed(lambda: model_opt(inp))[1])

eager: 0.33040280151367185


/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:135: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


compile: 32.55133984375


Notice that ``torch.compile`` takes a lot longer to complete
compared to eager. This is because ``torch.compile`` compiles
the model into optimized kernels as it executes. In our example, the
structure of the model doesn't change, and so recompilation is not
needed. So if we run our optimized model several more times, we should
see a significant improvement compared to eager.



In [9]:
eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, eager_time = timed(lambda: model(inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, compile_time = timed(lambda: model_opt(inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np
eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.015215616226196289
eager eval time 1: 0.014914560317993163
eager eval time 2: 0.015359999656677246
eager eval time 3: 0.014776320457458495
eager eval time 4: 0.014797696113586426
eager eval time 5: 0.014419808387756347
eager eval time 6: 0.012915871620178222
eager eval time 7: 0.012894047737121581
eager eval time 8: 0.017977344512939454
eager eval time 9: 0.017493152618408205
~~~~~~~~~~
compile eval time 0: 0.6544886474609375
compile eval time 1: 0.010012672424316407
compile eval time 2: 0.010078207969665527
compile eval time 3: 0.010185728073120117
compile eval time 4: 0.010267647743225097
compile eval time 5: 0.01029100799560547
compile eval time 6: 0.010919936180114746
compile eval time 7: 0.010456000328063965
compile eval time 8: 0.01011302375793457
compile eval time 9: 0.010038271903991699
~~~~~~~~~~
(eval) eager median: 0.014856128215789795, compile median: 0.010226687908172608, speedup: 1.452682271052546x
~~~~~~~~~~


And indeed, we can see that running our model with ``torch.compile``
results in a significant speedup. Speedup mainly comes from reducing Python overhead and
GPU read/writes, and so the observed speedup may vary on factors such as model
architecture and batch size. For example, if a model's architecture is simple
and the amount of data is large, then the bottleneck would be
GPU compute and the observed speedup may be less significant.

You may also see different speedup results depending on the chosen ``mode``
argument. The ``"reduce-overhead"`` mode uses CUDA graphs to further reduce
the overhead of Python. For your own models,
you may need to experiment with different modes to maximize speedup. You can
read more about modes [here](https://pytorch.org/get-started/pytorch-2.0/#user-experience)_.

You may might also notice that the second time we run our model with ``torch.compile`` is significantly
slower than the other runs, although it is much faster than the first run. This is because the ``"reduce-overhead"``
mode runs a few warm-up iterations for CUDA graphs.

For general PyTorch benchmarking, you can try using ``torch.utils.benchmark`` instead of the ``timed``
function we defined above. We wrote our own timing function in this tutorial to show
``torch.compile``'s compilation latency.

Now, let's consider comparing training.



In [10]:
model = init_model()
opt = torch.optim.Adam(model.parameters())

def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 0.6629071044921875
eager train time 1: 0.047333377838134766
eager train time 2: 0.043850624084472656
eager train time 3: 0.043911167144775394
eager train time 4: 0.04538675308227539
eager train time 5: 0.046142528533935544
eager train time 6: 0.0517283821105957
eager train time 7: 0.060922878265380856
eager train time 8: 0.04821811294555664
eager train time 9: 0.046341121673583986
~~~~~~~~~~


skipping cudagraphs due to input mutation


compile train time 0: 239.551015625
compile train time 1: 2.111662109375
compile train time 2: 0.046311424255371096
compile train time 3: 0.04087295913696289
compile train time 4: 0.04038326263427734
compile train time 5: 0.03729923248291016
compile train time 6: 0.03639683151245117
compile train time 7: 0.03693875122070313
compile train time 8: 0.03608876800537109
compile train time 9: 0.0361409912109375
~~~~~~~~~~
(train) eager median: 0.04683724975585937, compile median: 0.03884124755859375, speedup: 1.2058636810056962x
~~~~~~~~~~


Again, we can see that ``torch.compile`` takes longer in the first
iteration, as it must compile the model, but in subsequent iterations, we see
significant speedups compared to eager.

We remark that the speedup numbers presented in this tutorial are for
demonstration purposes only. Official speedup values can be seen at the
[TorchInductor performance dashboard](https://hud.pytorch.org/benchmark/compilers)_.



## Comparison to TorchScript and FX Tracing

We have seen that ``torch.compile`` can speed up PyTorch code.
Why else should we use ``torch.compile`` over existing PyTorch
compiler solutions, such as TorchScript or FX Tracing? Primarily, the
advantage of ``torch.compile`` lies in its ability to handle
arbitrary Python code with minimal changes to existing code.

One case that ``torch.compile`` can handle that other compiler
solutions struggle with is data-dependent control flow (the
``if x.sum() < 0:`` line below).



In [11]:
def f1(x, y):
    if x.sum() < 0:
        return -y
    return y

# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return torch.allclose(out1, out2)

inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)

TorchScript tracing ``f1`` results in
silently incorrect results, since only the actual control flow path
is traced.



In [12]:
traced_f1 = torch.jit.trace(f1, (inp1, inp2))
print("traced 1, 1:", test_fns(f1, traced_f1, (inp1, inp2)))
print("traced 1, 2:", test_fns(f1, traced_f1, (-inp1, inp2)))

traced 1, 1: True
traced 1, 2: False


/tmp/ipykernel_44172/4005623017.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:


FX tracing ``f1`` results in an error due to the presence of
data-dependent control flow.



In [13]:
import traceback as tb
try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_44172/1451191637.py", line 3, in <module>
    torch.fx.symbolic_trace(f1)
  File "/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/fx/_symbolic_trace.py", line 1150, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py", line 328, in _fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/_dynamo/external_utils.py", line 17, in inner
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/fx/_symbolic_trace.py", line 817, in trace
    (self.create_arg(fn(*args)),),
                     ^^^^^^^^^
  File "/tmp/ipykernel_44172/4005623017.py", line 2, in f1
    if x.sum() <

If we provide a value for ``x`` as we try to FX trace ``f1``, then
we run into the same problem as TorchScript tracing, as the data-dependent
control flow is removed in the traced function.



In [14]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
print("fx 1, 1:", test_fns(f1, fx_f1, (inp1, inp2)))
print("fx 1, 2:", test_fns(f1, fx_f1, (-inp1, inp2)))

fx 1, 1: True
fx 1, 2: False


/home/vanellope/miniforge3/envs/School/lib/python3.11/site-packages/torch/fx/_symbolic_trace.py:634: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


Now we can see that ``torch.compile`` correctly handles
data-dependent control flow.



In [15]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
print("compile 1, 1:", test_fns(f1, compile_f1, (inp1, inp2)))
print("compile 1, 2:", test_fns(f1, compile_f1, (-inp1, inp2)))
print("~" * 10)

compile 1, 1: True
compile 1, 2: True
~~~~~~~~~~


TorchScript scripting can handle data-dependent control flow, but this
solution comes with its own set of problems. Namely, TorchScript scripting
can require major code changes and will raise errors when unsupported Python
is used.

In the example below, we forget TorchScript type annotations and we receive
a TorchScript error because the input type for argument ``y``, an ``int``,
does not match with the default argument type, ``torch.Tensor``.



In [16]:
def f2(x, y):
    return x + y

inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_44172/895162831.py", line 9, in <module>
    script_f2(inp1, inp2)
RuntimeError: f2() Expected a value of type 'Tensor (inferred)' for argument 'y' but instead found type 'int'.
Inferred 'y' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 1
Value: 3
Declaration: f2(Tensor x, Tensor y) -> Tensor
Cast error details: Unable to cast 3 to Tensor


However, ``torch.compile`` is easily able to handle ``f2``.



In [17]:
compile_f2 = torch.compile(f2)
print("compile 2:", test_fns(f2, compile_f2, (inp1, inp2)))
print("~" * 10)

compile 2: True
~~~~~~~~~~


Another case that ``torch.compile`` handles well compared to
previous compilers solutions is the usage of non-PyTorch functions.



In [18]:
import scipy
def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

ModuleNotFoundError: No module named 'scipy'

TorchScript tracing treats results from non-PyTorch function calls
as constants, and so our results can be silently wrong.



In [ ]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", test_fns(f3, traced_f3, (inp2,)))

TorchScript scripting and FX tracing disallow non-PyTorch function calls.



In [ ]:
try:
    torch.jit.script(f3)
except:
    tb.print_exc()

try:
    torch.fx.symbolic_trace(f3)
except:
    tb.print_exc()

In comparison, ``torch.compile`` is easily able to handle
the non-PyTorch function call.



In [ ]:
compile_f3 = torch.compile(f3)
print("compile 3:", test_fns(f3, compile_f3, (inp2,)))

## TorchDynamo and FX Graphs

One important component of ``torch.compile`` is TorchDynamo.
TorchDynamo is responsible for JIT compiling arbitrary Python code into
[FX graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph)_, which can
then be further optimized. TorchDynamo extracts FX graphs by analyzing Python bytecode
during runtime and detecting calls to PyTorch operations.

Normally, TorchInductor, another component of ``torch.compile``,
further compiles the FX graphs into optimized kernels,
but TorchDynamo allows for different backends to be used. In order to inspect
the FX graphs that TorchDynamo outputs, let us create a custom backend that
outputs the FX graph and simply returns the graph's unoptimized forward method.



In [ ]:
from typing import List
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

Using our custom backend, we can now see how TorchDynamo is able to handle
data-dependent control flow. Consider the function below, where the line
``if b.sum() < 0`` is the source of data-dependent control flow.



In [ ]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

The output reveals that TorchDynamo extracted 3 different FX graphs
corresponding the following code (order may differ from the output above):

1. ``x = a / (torch.abs(a) + 1)``
2. ``b = b * -1; return x * b``
3. ``return x * b``

When TorchDynamo encounters unsupported Python features, such as data-dependent
control flow, it breaks the computation graph, lets the default Python
interpreter handle the unsupported code, then resumes capturing the graph.

Let's investigate by example how TorchDynamo would step through ``bar``.
If ``b.sum() < 0``, then TorchDynamo would run graph 1, let
Python determine the result of the conditional, then run
graph 2. On the other hand, if ``not b.sum() < 0``, then TorchDynamo
would run graph 1, let Python determine the result of the conditional, then
run graph 3.

This highlights a major difference between TorchDynamo and previous PyTorch
compiler solutions. When encountering unsupported Python features,
previous solutions either raise an error or silently fail.
TorchDynamo, on the other hand, will break the computation graph.

We can see where TorchDynamo breaks the graph by using ``torch._dynamo.explain``:



In [ ]:
# Reset since we are using a different backend.
torch._dynamo.reset()
explain_output = torch._dynamo.explain(bar)(torch.randn(10), torch.randn(10))
print(explain_output)

In order to maximize speedup, graph breaks should be limited.
We can force TorchDynamo to raise an error upon the first graph
break encountered by using ``fullgraph=True``:



In [ ]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

And below, we demonstrate that TorchDynamo does not break the graph on
the model we used above for demonstrating speedups.



In [ ]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

We can use ``torch.export`` (from PyTorch 2.1+) to extract a single, exportable
FX graph from the input PyTorch program. The exported graph is intended to be
run on different (i.e. Python-less) environments. One important restriction
is that the ``torch.export`` does not support graph breaks. Please check
[this tutorial](https://pytorch.org/tutorials/intermediate/torch_export_tutorial.html)_
for more details on ``torch.export``.



## Conclusion

In this tutorial, we introduced ``torch.compile`` by covering
basic usage, demonstrating speedups over eager mode, comparing to previous
PyTorch compiler solutions, and briefly investigating TorchDynamo and its interactions
with FX graphs. We hope that you will give ``torch.compile`` a try!

